In [7]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo
from astropy.table import Table
import matplotlib.pyplot as plt
import warnings

project_root = Path.cwd().parent
print(f"Project root: {project_root}")

Project root: /Users/david/Code/msc


In [8]:
# User input: run folder name
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

lightcurve_files = sorted(run_folder.glob("*_lightcurve.csv"))
print(f"Found {len(lightcurve_files)} lightcurve CSV in {run_folder}")
for f in lightcurve_files:
    print(f"{f.name}")

Found 9 lightcurve CSV in /Users/david/Code/msc/runs/test2
ZTF19aaagfme_lightcurve.csv
ZTF19aaairqh_lightcurve.csv
ZTF19aaarhtg_lightcurve.csv
ZTF19aabmybj_lightcurve.csv
ZTF19aabvfwn_lightcurve.csv
ZTF19aabyheu_lightcurve.csv
ZTF19aabyppp_lightcurve.csv
ZTF19aabyuze_lightcurve.csv
ZTF19aacgslb_lightcurve.csv


In [9]:
ztf_cleansed_path = project_root / "ztf_cleansed.csv"
ztf_df = pd.read_csv(ztf_cleansed_path)

# Initialize list to collect results
results_list = []

for idx, lc_path in enumerate(lightcurve_files, 1):
    print(f"Processing file {idx}/{len(lightcurve_files)}: {lc_path.name}")
    lc_df = pd.read_csv(lc_path)

    # clean
    lc_df['MJD'] = pd.to_numeric(lc_df['MJD'], errors='coerce')
    lc_df = lc_df.dropna(subset=['MJD', 'filter', 'forced_ujy', 'forced_ujy_error'])
    lc_df['filter'] = lc_df['filter'].astype(str).str.strip().str.lower()
    lc_df = lc_df[lc_df['forced_ujy'] > 0]
    lc_df = lc_df[lc_df['forced_ujy_error'] > 0]
    lc_df = lc_df.sort_values('MJD')

    bands = lc_df['filter'].map({'g': 'ztfg', 'r': 'ztfr'}).values

    data = Table({
        'time': lc_df['MJD'].values,
        'band': bands,
        'flux': lc_df['forced_ujy'].values,
        'fluxerr': lc_df['forced_ujy_error'].values,
        'zp': np.full(len(lc_df), 23.9),
        'zpsys': np.array(['ab'] * len(lc_df)),
    })

    print(f"  Loaded {len(data)} data points after cleaning.")

    model = sncosmo.Model(source='salt3')
    obj_id = lc_df['ztf_id'].iloc[0]
    z_ztf = ztf_df.loc[ztf_df['ZTFID'] == obj_id, 'host_redshift'].iloc[0]
    model.set(z=z_ztf)
    result, fitted_model = sncosmo.fit_lc(
        data, model,
        ['t0', 'x0', 'x1', 'c'])

    # Collect results (param order is model.param_names, e.g. ['z','t0','x0','x1','c'])
    params = dict(zip(result.param_names, result.parameters))
    result_dict = {
        'ztf_id': obj_id,
        'redshift': z_ztf,
        'ncall': result.ncall,
        'ndof': result.ndof,
        'chisq': result.chisq,
        't0': params['t0'],
        'x0': params['x0'],
        'x1': params['x1'],
        'c': params['c']
    }
    results_list.append(result_dict)

    print(f"  Fit completed for {obj_id}")

# After loop, create dataframe and save to CSV
results_df = pd.DataFrame(results_list)
output_csv_path = run_folder / "sncosmo_fits.csv"
results_df.to_csv(output_csv_path, index=False)
print(f"Results saved to {output_csv_path}")

Processing file 1/9: ZTF19aaagfme_lightcurve.csv
  Loaded 57 data points after cleaning.
  Fit completed for ZTF19aaagfme
Processing file 2/9: ZTF19aaairqh_lightcurve.csv
  Loaded 30 data points after cleaning.
  Fit completed for ZTF19aaairqh
Processing file 3/9: ZTF19aaarhtg_lightcurve.csv
  Loaded 25 data points after cleaning.
  Fit completed for ZTF19aaarhtg
Processing file 4/9: ZTF19aabmybj_lightcurve.csv
  Loaded 24 data points after cleaning.
  Fit completed for ZTF19aabmybj
Processing file 5/9: ZTF19aabvfwn_lightcurve.csv
  Loaded 9 data points after cleaning.
  Fit completed for ZTF19aabvfwn
Processing file 6/9: ZTF19aabyheu_lightcurve.csv
  Loaded 46 data points after cleaning.
  Fit completed for ZTF19aabyheu
Processing file 7/9: ZTF19aabyppp_lightcurve.csv
  Loaded 44 data points after cleaning.
  Fit completed for ZTF19aabyppp
Processing file 8/9: ZTF19aabyuze_lightcurve.csv
  Loaded 49 data points after cleaning.
  Fit completed for ZTF19aabyuze
Processing file 9/9: ZTF1